<a href="https://colab.research.google.com/github/shrikant78/DSSummit/blob/master/Fake_news_Tuning_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)

In [2]:
pwd

'/content'

In [3]:
real = pd.read_csv("/content/RealData.csv")
fake = pd.read_csv("/content/FakeData.csv")

In [4]:
fake['Authenticity'] = 'Fake'
real['Authenticity'] = 'Real'
news_data = fake.append(real)
news_data.head()

,text,Authenticity
0,"Breaking: At least 10 dead, 5 injured after tO gunman open fire in offices of Charlie Hebdo,satirical mag that published Mohammed cartoons",Fake
1,"France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58",Fake
2,"Ten killed in shooting at headquarters of French satirical weekly Charlie Hebdo, says French media citing witnesses #c4news",Fake
3,BREAKING: 10 dead in shooting at headquarters of French satirical weekly #CharlieHebdo - French media citing witnesses. #France,Fake
4,Reuters: 10 people shot dead at headquarters of French satirical weekly publication Charlie Hebdo in #Paris after gunmen stormed the office,Fake


In [5]:
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
#nltk.download('wordnet')

In [6]:
sw = stopwords.words('english')
new_words=('’','“', '”','€','@','#','™','\x9d','â')
for i in new_words:
    sw.append(i)

In [7]:
from nltk.stem.porter import *
from autocorrect import Speller
from nltk.tag import pos_tag

spell = Speller(lang='en')

lemmatizer = nltk.stem.WordNetLemmatizer()
porterStemmer = PorterStemmer()
tknzr = TweetTokenizer()


def removeProperNouns(text):
    tagged_text = pos_tag(text.split())
    sanspropernouns = [word for word,pos in tagged_text if pos != 'NNP']
    sep = ' '
    returnVal = sep.join(sanspropernouns)
    return returnVal

def stemming_text(text):
    stemWords = [porterStemmer.stem(word) for word in text]
    return stemWords

def lemmatize_text(text):   
    return [lemmatizer.lemmatize(w) for w in text]

def spell_tokenize_text(text):
    spells = [spell(w) for w in (tknzr.tokenize(text))]
    return spells

def tokenize_text(text):
    tokenized_text = tknzr.tokenize(text)
    return tokenized_text

def remove_numerals(text):
    retText = ''.join(c for c in text if not c.isdigit())
    return retText

In [8]:
#pip install autocorrect

In [9]:
#Remove Digits
news_data['textSansNum'] = news_data['text'].str.replace('\d+', '')
# Convert to lower case
news_data['textSansNum'] = news_data['textSansNum'].str.lower()

# Tokenizing, Lemmatizing and stemming
news_data['tokenized_text'] = news_data['textSansNum'].apply(tokenize_text)
news_data['lemmatized_text'] = news_data['tokenized_text'].apply(lemmatize_text)
#news_data['stemmed_text'] = news_data['tokenized_text'].apply(stemming_text)
# Remove stopwords
#news_data['filtered_text'] = news_data['stemmed_text'].apply(lambda x: [item for item in x if item not in sw])
news_data['filtered_text'] = news_data['lemmatized_text'].apply(lambda x: [item for item in x if item not in sw])
# Remove punctuation
from string import punctuation
news_data['filtered_text'] = news_data['filtered_text'].apply(lambda x: [item for item in x if item not in punctuation])
# Join Text 
news_data['joined_text'] = news_data['filtered_text'].apply(lambda x: ' '.join(map(str, x)))
#text = " ".join(text for text in news_data.joined_text) 

In [10]:
news_data.head()

,text,Authenticity,textSansNum,tokenized_text,lemmatized_text,filtered_text,joined_text
0,"Breaking: At least 10 dead, 5 injured after tO gunman open fire in offices of Charlie Hebdo,satirical mag that published Mohammed cartoons",Fake,"breaking: at least dead, injured after to gunman open fire in offices of charlie hebdo,satirical mag that published mohammed cartoons","[breaking, :, at, least, dead, ,, injured, after, to, gunman, open, fire, in, offices, of, charlie, hebdo, ,, satirical, mag, that, published, mohammed, cartoons]","[breaking, :, at, least, dead, ,, injured, after, to, gunman, open, fire, in, office, of, charlie, hebdo, ,, satirical, mag, that, published, mohammed, cartoon]","[breaking, least, dead, injured, gunman, open, fire, office, charlie, hebdo, satirical, mag, published, mohammed, cartoon]",breaking least dead injured gunman open fire office charlie hebdo satirical mag published mohammed cartoon
1,"France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58",Fake,"france: people dead after shooting at hq of satirical weekly newspaper #charliehebdo, according to witnesses http://t.co/fkyxgmus","[france, :, people, dead, after, shooting, at, hq, of, satirical, weekly, newspaper, #charliehebdo, ,, according, to, witnesses, http://t.co/fkyxgmus]","[france, :, people, dead, after, shooting, at, hq, of, satirical, weekly, newspaper, #charliehebdo, ,, according, to, witness, http://t.co/fkyxgmus]","[france, people, dead, shooting, hq, satirical, weekly, newspaper, #charliehebdo, according, witness, http://t.co/fkyxgmus]",france people dead shooting hq satirical weekly newspaper #charliehebdo according witness http://t.co/fkyxgmus
2,"Ten killed in shooting at headquarters of French satirical weekly Charlie Hebdo, says French media citing witnesses #c4news",Fake,"ten killed in shooting at headquarters of french satirical weekly charlie hebdo, says french media citing witnesses #cnews","[ten, killed, in, shooting, at, headquarters, of, french, satirical, weekly, charlie, hebdo, ,, says, french, media, citing, witnesses, #cnews]","[ten, killed, in, shooting, at, headquarters, of, french, satirical, weekly, charlie, hebdo, ,, say, french, medium, citing, witness, #cnews]","[ten, killed, shooting, headquarters, french, satirical, weekly, charlie, hebdo, say, french, medium, citing, witness, #cnews]",ten killed shooting headquarters french satirical weekly charlie hebdo say french medium citing witness #cnews
3,BREAKING: 10 dead in shooting at headquarters of French satirical weekly #CharlieHebdo - French media citing witnesses. #France,Fake,breaking: dead in shooting at headquarters of french satirical weekly #charliehebdo - french media citing witnesses. #france,"[breaking, :, dead, in, shooting, at, headquarters, of, french, satirical, weekly, #charliehebdo, -, french, media, citing, witnesses, ., #france]","[breaking, :, dead, in, shooting, at, headquarters, of, french, satirical, weekly, #charliehebdo, -, french, medium, citing, witness, ., #france]","[breaking, dead, shooting, headquarters, french, satirical, weekly, #charliehebdo, french, medium, citing, witness, #france]",breaking dead shooting headquarters french satirical weekly #charliehebdo french medium citing witness #france
4,Reuters: 10 people shot dead at headquarters of French satirical weekly publication Charlie Hebdo in #Paris after gunmen stormed the office,Fake,reuters: people shot dead at headquarters of french satirical weekly publication charlie hebdo in #paris after gunmen stormed the office,"[reuters, :, people, shot, dead, at, headquarters, of, french, satirical, weekly, publication, charlie, hebdo, in, #paris, after, gunmen, stormed, the, office]","[reuters, :, people, shot, dead, at, headquarters, of, french, satirical, weekly, publication, charlie, hebdo, in, #paris, after, gunman, stormed, the, office]","[reuters, people, shot, dead, headquarters, french, satiric

In [11]:
# code below is added to tune the model with LogisticRegression, User Input and Pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
import pickle 
#import mglearn
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix,classification_report,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [12]:
#Logistics Regression without tuning

#this should run on the cleaned data and final joined text, it was runnign on _news_data.text only which was original text only
vectorizer = TfidfVectorizer(stop_words=sw,lowercase=True)
y = news_data.Authenticity
x = vectorizer.fit_transform(news_data.joined_text)
print(x.shape)
print(y.shape)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

LR_classifier = LogisticRegression()

LR_classifier.fit(X_train, y_train)
LR_predictions = LR_classifier.predict(X_test)

print(confusion_matrix(y_test, LR_predictions))
print(classification_report(y_test,LR_predictions))

(74823, 131428)
(74823,)
[[ 8156  1364]
 [  548 12379]]
              precision    recall  f1-score   support

        Fake       0.94      0.86      0.90      9520
        Real       0.90      0.96      0.93     12927

    accuracy                           0.91     22447
   macro avg       0.92      0.91      0.91     22447
weighted avg       0.92      0.91      0.91     22447



In [13]:
#Logistics Regression with tuning
y = news_data.Authenticity
x = news_data.joined_text
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

start_time=time.time()

pipe = make_pipeline(TfidfVectorizer(), OneVsRestClassifier(LogisticRegression()))
param_grid = {'tfidfvectorizer__max_features': [30000, 50000, 60000],
              'onevsrestclassifier__estimator__solver': ['liblinear', 'sag']
             } 
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc')
#roc_auc
grid3 = grid.fit(X_train, y_train)

end_time=time.time()
print("total time",end_time-start_time)

total time 295.3961567878723


In [14]:
predictions = grid3.predict(X_test)

print(confusion_matrix(y_test, predictions))
print(classification_report(y_test,predictions))

[[ 8186  1341]
 [  558 12362]]
              precision    recall  f1-score   support

        Fake       0.94      0.86      0.90      9527
        Real       0.90      0.96      0.93     12920

    accuracy                           0.92     22447
   macro avg       0.92      0.91      0.91     22447
weighted avg       0.92      0.92      0.91     22447



In [15]:
print(grid3.best_params_)

{'onevsrestclassifier__estimator__solver': 'liblinear', 'tfidfvectorizer__max_features': 50000}


In [16]:
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import kneighbors_graph

# use tfidf to transform texts into feature vectors
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(news_data.joined_text)



In [19]:
N = vectors.shape[0]
#print(vectors)

  (0, 16141)	0.2974038955129988
  (0, 70380)	0.29101077102122713
  (0, 86535)	0.21605490376786723
  (0, 64724)	0.398648170259761
  (0, 95828)	0.34465737088581366
  (0, 45820)	0.28539658397880857
  (0, 17596)	0.26443056085733885
  (0, 77397)	0.14604877867521335
  (0, 36940)	0.20522720915740808
  (0, 78231)	0.18039820933698614
  (0, 43778)	0.2576181675318418
  (0, 51247)	0.252216806730244
  (0, 25087)	0.21573743162935002
  (0, 61083)	0.17127048225159597
  (0, 13138)	0.22192681591521563
  (1, 37208)	0.5293480818765798
  (1, 19804)	0.1461139258001439
  (1, 48353)	0.17921191798553823
  (1, 120263)	0.2436146864147075
  (1, 795)	0.1389768157975084
  (1, 17604)	0.23741017732276598
  (1, 74273)	0.21990780416405067
  (1, 118774)	0.28418341860150576
  (1, 48197)	0.36482331724881345
  (1, 99148)	0.20935446968587776
  :	:
  (74822, 34551)	0.09632063138370996
  (74822, 23741)	0.05567928459433371
  (74822, 103302)	0.05540111521525844
  (74822, 103325)	0.03794092515686154
  (74822, 114084)	0.049964705

In [ ]:
feature_names = vectorizer.get_feature_names()
print(vectorizer.get_feature_names())
print(vectors.toarray())
#print(pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())



['__', '___', '___adolf', '___featured', '___image', '___photo', '___q__', '___supportsdonaldtrump', '__mref', '__saada', '_a_w_w_r_', '_ad_container', '_all_', '_americasidiot', '_anyone_', '_batou_', '_bigger_', '_brendancarroll', '_brittanyv', '_cappex', '_cillaw', '_cingraham', '_elizabethmay', '_fntns_ega', '_gop', '_habibilicious_', '_impossible', '_js', '_julian_nyc', '_juliannemoore', '_julmplporw', '_kcjqqynq', '_kimvfrsps', '_known_', '_lpvv', '_lzlngexyk', '_makada_', '_maryali', '_mdiaz', '_michaelhughes', '_mzckprs', '_mzckprscamerota', '_mzckprsif', '_nackt', '_nehoda_', '_nevertrump', '_nkolhiuthis', '_now_', '_nwifyy', '_o', '_only_', '_peppercats', '_philosophica', '_pkrqjq', '_r', '_robespierre__', '_sgsiuw', '_stephenward', '_targetsmart', '_terrible_', '_thespeakeasy', '_tonybriscoe', '_trump', '_under_current', '_westwest_yall', '_wintergirl', '_your', '_zsixpifyrex', 'a_h_goldstein', 'a_n_t_s_', 'a_p_crowley', 'a_r_marshall', 'aa', 'aaa', 'aaaaaaaand', 'aaaaackkk'

In [20]:
print(feature_names)

['__', '___', '___adolf', '___featured', '___image', '___photo', '___q__', '___supportsdonaldtrump', '__mref', '__saada', '_a_w_w_r_', '_ad_container', '_all_', '_americasidiot', '_anyone_', '_batou_', '_bigger_', '_brendancarroll', '_brittanyv', '_cappex', '_cillaw', '_cingraham', '_elizabethmay', '_fntns_ega', '_gop', '_habibilicious_', '_impossible', '_js', '_julian_nyc', '_juliannemoore', '_julmplporw', '_kcjqqynq', '_kimvfrsps', '_known_', '_lpvv', '_lzlngexyk', '_makada_', '_maryali', '_mdiaz', '_michaelhughes', '_mzckprs', '_mzckprscamerota', '_mzckprsif', '_nackt', '_nehoda_', '_nevertrump', '_nkolhiuthis', '_now_', '_nwifyy', '_o', '_only_', '_peppercats', '_philosophica', '_pkrqjq', '_r', '_robespierre__', '_sgsiuw', '_stephenward', '_targetsmart', '_terrible_', '_thespeakeasy', '_tonybriscoe', '_trump', '_under_current', '_westwest_yall', '_wintergirl', '_your', '_zsixpifyrex', 'a_h_goldstein', 'a_n_t_s_', 'a_p_crowley', 'a_r_marshall', 'aa', 'aaa', 'aaaaaaaand', 'aaaaackkk'

In [21]:
print(corpus_index)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df = pd.DataFrame(vectors.T.todense(), index=feature_names, columns=corpus_index)
print(df)

In [ ]:
# build the graph which is full-connected

mat = kneighbors_graph(vectors, N, metric='cosine', mode='distance', include_self=True)
mat.data = 1 - mat.data  # to similarity



In [2]:
g = nx.from_scipy_sparse_matrix(mat, create_using=nx.Graph())

NameError: ignored